<a href="https://colab.research.google.com/github/Sarah-0405/Bachelorarbeit_/blob/main/Gitterzellen__auf__Stadtgebiet_zuschneiden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


In [ ]:
# Schritt 1: CSV-Datei mit Gitterzellen laden
# Annahme: CSV enthält Spalten "X" und "Y" für die Koordinaten
csv_path = "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Durchschnittsalter_in_Gitterzellen/Zensus2022_Durchschnittsalter_100m-Gitter.csv"
gitter_df = pd.read_csv(csv_path, sep=";")

print(gitter_df.head)

<ipython-input-2-abe5adacc2ce>:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  gitter_df = pd.read_csv(csv_path, sep=";")


<bound method NDFrame.head of                          GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0        CRS3035RES100mN2689100E4337000    4337050    2689150   
1        CRS3035RES100mN2689100E4341100    4341150    2689150   
2        CRS3035RES100mN2690800E4341200    4341250    2690850   
3        CRS3035RES100mN2691200E4341200    4341250    2691250   
4        CRS3035RES100mN2691300E4341200    4341250    2691350   
...                                 ...        ...        ...   
3088031  CRS3035RES100mN3547500E4220100    4220150    3547550   
3088032  CRS3035RES100mN3547600E4219800    4219850    3547650   
3088033  CRS3035RES100mN3547600E4219900    4219950    3547650   
3088034  CRS3035RES100mN3547600E4220000    4220050    3547650   
3088035  CRS3035RES100mN3547600E4220100    4220150    3547650   

        Durchschnittsalter werterlaeuternde_Zeichen  
0                    36,75                      NaN  
1                    39,78                      NaN  
2                    37,56 

In [ ]:
# Lade die CSV-Datei und drucke die Spaltennamen
print("Spaltennamen in der CSV-Datei:", gitter_df.columns)


Spaltennamen in der CSV-Datei: Index(['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'Durchschnittsalter',
       'werterlaeuternde_Zeichen'],
      dtype='object')


In [ ]:
# Assuming your DataFrame is named 'gitter_df'
gitter_df = gitter_df.drop(columns=["werterlaeuternde_Zeichen"])

# Print the remaining columns to verify
print("Spaltennamen nach dem Löschen:", gitter_df.columns)

Spaltennamen nach dem Löschen: Index(['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'Durchschnittsalter'], dtype='object')


In [ ]:
print(gitter_df.head)

<bound method NDFrame.head of                          GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0        CRS3035RES100mN2689100E4337000    4337050    2689150   
1        CRS3035RES100mN2689100E4341100    4341150    2689150   
2        CRS3035RES100mN2690800E4341200    4341250    2690850   
3        CRS3035RES100mN2691200E4341200    4341250    2691250   
4        CRS3035RES100mN2691300E4341200    4341250    2691350   
...                                 ...        ...        ...   
3088031  CRS3035RES100mN3547500E4220100    4220150    3547550   
3088032  CRS3035RES100mN3547600E4219800    4219850    3547650   
3088033  CRS3035RES100mN3547600E4219900    4219950    3547650   
3088034  CRS3035RES100mN3547600E4220000    4220050    3547650   
3088035  CRS3035RES100mN3547600E4220100    4220150    3547650   

        Durchschnittsalter  
0                    36,75  
1                    39,78  
2                    37,56  
3                    38,05  
4                    49,64  
...            

In [ ]:
# Annahme: CSV enthält Spalten "x_mp_100m" und "y_mp_100m" für die Koordinaten

print("Spaltennamen in der CSV-Datei:", gitter_df.columns)  # Debugging

# Geometrie aus den Koordinaten erstellen
gitter_df['geometry'] = gitter_df.apply(
    lambda row: Point(row['x_mp_100m'], row['y_mp_100m']), axis=1
)
gitter_gdf = gpd.GeoDataFrame(gitter_df, geometry='geometry', crs="EPSG:3035")  # EPSG:3035 = ETRS89-LAEA Europe

# Schritt 2: Stadtgrenze von Nördlingen laden (z. B. Shapefile)
stadt_path = "/content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Nördlingen_Gerenze_shp.shp"
stadt_gdf = gpd.read_file(stadt_path)

# Schritt 3: Prüfen, ob beide Layer dasselbe CRS verwenden
if gitter_gdf.crs != stadt_gdf.crs:
    stadt_gdf = stadt_gdf.to_crs(gitter_gdf.crs)

# Schritt 4: Zuschneiden der Gitterzellen auf die Stadtgrenze
gitter_in_stadt = gitter_gdf[gitter_gdf.geometry.within(stadt_gdf.union_all())]

# Schritt 5: Ergebnis speichern
output_path = "gitterzellen_noerdlingen.csv"
gitter_in_stadt.to_csv(output_path, index=False)

print(f"Ergebnis gespeichert: {output_path}")


Spaltennamen in der CSV-Datei: Index(['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'Durchschnittsalter'], dtype='object')


<ipython-input-6-8de778748d13>:20: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gitter_in_stadt = gitter_gdf[gitter_gdf.geometry.within(stadt_gdf.unary_union)]


Ergebnis gespeichert: gitterzellen_noerdlingen.csv


In [ ]:
# Read the saved CSV file into a DataFrame
output = pd.read_csv(output_path) # Read the saved file
print(output) # Call the head() method on the DataFrame

                     GITTER_ID_100m  x_mp_100m  y_mp_100m Durchschnittsalter  \
0    CRS3035RES100mN2854400E4359000    4359050    2854450              37,13   
1    CRS3035RES100mN2854500E4358400    4358450    2854550              53,48   
2    CRS3035RES100mN2854500E4358700    4358750    2854550              57,78   
3    CRS3035RES100mN2854500E4358800    4358850    2854550              43,04   
4    CRS3035RES100mN2854500E4358900    4358950    2854550              36,65   
..                              ...        ...        ...                ...   
792  CRS3035RES100mN2866000E4361200    4361250    2866050              55,21   
793  CRS3035RES100mN2866000E4361300    4361350    2866050              28,85   
794  CRS3035RES100mN2866000E4361500    4361550    2866050              57,18   
795  CRS3035RES100mN2866100E4361300    4361350    2866150              62,33   
796  CRS3035RES100mN2866400E4361700    4361750    2866450              44,00   

                    geometry  
0    POI

In [ ]:
# Lade die neue CSV-Datei
csv_path = "/content/gitterzellen_noerdlingen.csv"
df = pd.read_csv(csv_path)

# Spaltennamen anzeigen (optional, um die Namen zu prüfen)
print("Spaltennamen vor dem Löschen:", df.columns)


Spaltennamen vor dem Löschen: Index(['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'Durchschnittsalter',
       'geometry'],
      dtype='object')


In [ ]:
# Lösche die Spalte "Durchschnittsalter"
df = df.drop(columns=["Durchschnittsalter"])

# Spaltennamen nach dem Löschen prüfen
print("Spaltennamen nach dem Löschen:", df.columns)


Spaltennamen nach dem Löschen: Index(['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'geometry'], dtype='object')


In [ ]:
# Bereinigte Tabelle als neue CSV speichern
output_path = "gitterzellen_noerdlingen_clean.csv"
df.to_csv(output_path, index=False)

print(f"Bereinigte CSV-Datei gespeichert unter: {output_path}")


Bereinigte CSV-Datei gespeichert unter: gitterzellen_noerdlingen_clean.csv


In [ ]:
# Read the saved CSV file into a DataFrame
output = pd.read_csv(output_path) # Read the saved file
print(output) # Call the head() method on the DataFrame

                     GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0    CRS3035RES100mN2854400E4359000    4359050    2854450   
1    CRS3035RES100mN2854500E4358400    4358450    2854550   
2    CRS3035RES100mN2854500E4358700    4358750    2854550   
3    CRS3035RES100mN2854500E4358800    4358850    2854550   
4    CRS3035RES100mN2854500E4358900    4358950    2854550   
..                              ...        ...        ...   
792  CRS3035RES100mN2866000E4361200    4361250    2866050   
793  CRS3035RES100mN2866000E4361300    4361350    2866050   
794  CRS3035RES100mN2866000E4361500    4361550    2866050   
795  CRS3035RES100mN2866100E4361300    4361350    2866150   
796  CRS3035RES100mN2866400E4361700    4361750    2866450   

                    geometry  
0    POINT (4359050 2854450)  
1    POINT (4358450 2854550)  
2    POINT (4358750 2854550)  
3    POINT (4358850 2854550)  
4    POINT (4358950 2854550)  
..                       ...  
792  POINT (4361250 2866050)  
793  POINT (4361350

Gitterzellen jetzt auf VWG Nördlingen zugeschnitten

jetzt: Workflow auf weitere Städte anwenden => reproduzierbar machen

erstmal: shp.-Datei auf andere Städte zuschneiden: Kempten und Ingolstadt

mit existierendem Shapefile (in qgis extrahiert) eine csv. Tabelle auf dieses zuschneiden

In [ ]:
# Funktion: Gitterzellen auf Stadt zuschneiden
def zuschneiden_gitterzellen(csv_path, shapefile_path, output_path, drop_columns=None):
    """
    Schneidet Gitterzellen auf die Stadtgrenze zu und speichert das Ergebnis.

    Args:
        csv_path (str): Pfad zur CSV-Datei mit Gitterzellen.
        shapefile_path (str): Pfad zur Shapefile-Datei der Stadtgrenze.
        output_path (str): Pfad, um die bereinigte CSV zu speichern.
        drop_columns (list, optional): Liste der Spalten, die gelöscht werden sollen.
    """
    # Schritt 1: CSV-Datei mit Gitterzellen laden
    gitter_df = pd.read_csv(csv_path, sep=";")
    gitter_df['geometry'] = gitter_df.apply(
        lambda row: Point(row['x_mp_100m'], row['y_mp_100m']), axis=1
    )
    gitter_gdf = gpd.GeoDataFrame(gitter_df, geometry='geometry', crs="EPSG:3035")

    # Schritt 2: Stadtgrenze laden
    stadt_gdf = gpd.read_file(shapefile_path)

    # Schritt 3: Prüfen, ob beide Layer dasselbe CRS verwenden
    if gitter_gdf.crs != stadt_gdf.crs:
        stadt_gdf = stadt_gdf.to_crs(gitter_gdf.crs)

    # Schritt 4: Zuschneiden der Gitterzellen auf die Stadtgrenze
    gitter_in_stadt = gitter_gdf[gitter_gdf.geometry.within(stadt_gdf.union_all())]

    # Schritt 5: Optional: Spalten löschen: da als CSV-Datei mit Gitterzellen die Datei "Durchschnittsalter_100m_Gitter" verwendet wurde, wurde im folgenden die Spalte "Durchschnittsalter" gelöscht, um lediglich die Geometriedaten zu erhalten.
    if drop_columns:
        gitter_in_stadt = gitter_in_stadt.drop(columns=drop_columns)

    # Schritt 6: Ergebnis speichern
    gitter_in_stadt.to_csv(output_path, index=False)
    print(f"Ergebnis gespeichert: {output_path}")

    # Tabelle anzeigen (nur die ersten 5 Zeilen)
    print(f"Ergebnisse für {shapefile_path} (erste 5 Zeilen):")
    print(gitter_in_stadt.head())  # Zeigt die ersten 5 Zeilen der bereinigten Tabelle an


# Beispiel: Zuschnitt auf zwei Städte
csv_path = "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Durchschnittsalter_in_Gitterzellen/Zensus2022_Durchschnittsalter_100m-Gitter.csv"  # CSV mit Gitterzellen
Kempten_shapefile = "/content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Kempten (Allgäu) .shp"  # Shapefile für Stadt 1
Ingolstadt_shapefile = "/content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Ingolstadt .shp"  # Shapefile für Stadt 2

# Zuschnitt für Stadt 1
zuschneiden_gitterzellen(
    csv_path=csv_path,
    shapefile_path=Kempten_shapefile,
    output_path="gitterzellen_kempten_clean.csv",
    drop_columns=["Durchschnittsalter", "werterlaeuternde_Zeichen"]  # Entferne unnötige Spalten
)

# Zuschnitt für Stadt 2
zuschneiden_gitterzellen(
    csv_path=csv_path,
    shapefile_path=Ingolstadt_shapefile,
    output_path="gitterzellen_Ingolstadt_clean.csv",
    drop_columns=["Durchschnittsalter", "werterlaeuternde_Zeichen"]
)



<ipython-input-5-3857b2ff0c94>:17: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  gitter_df = pd.read_csv(csv_path, sep=";")


Ergebnis gespeichert: gitterzellen_kempten_clean.csv
Ergebnisse für /content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Kempten (Allgäu) .shp (erste 5 Zeilen):
                       GITTER_ID_100m  x_mp_100m  y_mp_100m  \
38078  CRS3035RES100mN2731600E4345000    4345050    2731650   
38399  CRS3035RES100mN2731700E4344700    4344750    2731750   
38721  CRS3035RES100mN2731800E4342000    4342050    2731850   
38722  CRS3035RES100mN2731800E4344600    4344650    2731850   
38723  CRS3035RES100mN2731800E4344700    4344750    2731850   

                      geometry  
38078  POINT (4345050 2731650)  
38399  POINT (4344750 2731750)  
38721  POINT (4342050 2731850)  
38722  POINT (4344650 2731850)  
38723  POINT (4344750 2731850)  


<ipython-input-5-3857b2ff0c94>:17: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  gitter_df = pd.read_csv(csv_path, sep=";")


Ergebnis gespeichert: gitterzellen_Ingolstadt_clean.csv
Ergebnisse für /content/drive/MyDrive/Daten BA /Shapefile_Zensus2022/EPSG_25832/Ingolstadt .shp (erste 5 Zeilen):
                        GITTER_ID_100m  x_mp_100m  y_mp_100m  \
452589  CRS3035RES100mN2843300E4424300    4424350    2843350   
454439  CRS3035RES100mN2843700E4425900    4425950    2843750   
454440  CRS3035RES100mN2843700E4426000    4426050    2843750   
454917  CRS3035RES100mN2843800E4423200    4423250    2843850   
454918  CRS3035RES100mN2843800E4423300    4423350    2843850   

                       geometry  
452589  POINT (4424350 2843350)  
454439  POINT (4425950 2843750)  
454440  POINT (4426050 2843750)  
454917  POINT (4423250 2843850)  
454918  POINT (4423350 2843850)  


erfolgreich Gitterzellen auf die 3 Städte Nördlingen, Kempten und Ingolstadt geclippt und als csv. gespeichert. Die Tabellen können jetzt räumlich analysiert werden

nächster Schritt: den Tabellen die Spalten für die zu vergleichenden Werte hinzufügen (Durchschnittsalter,...)


In [ ]:
# Funktion: Merkmalsattribute zu Hauptdokument hinzufügen
def kombiniere_daten(hauptdokument_path, merkmale_paths, output_path, sep=";"):
    """
    Kombiniert ein Hauptdokument (Gitterzellen einer Stadt) mit Merkmalsattributen.

    Args:
        hauptdokument_path (str): Pfad zum Hauptdokument (Gitterzellen für eine Stadt).
        merkmale_paths (list): Liste der Pfade zu den CSV-Dateien mit Attributen.
        output_path (str): Pfad zur kombinierten und bereinigten CSV-Datei.
        sep (string, optional): Trennzeichen der csv. Datei
    """
    # Hauptdokument laden (z. B. Gitterzellen für Nördlingen)
    hauptdokument = pd.read_csv(hauptdokument_path)

    # Iteriere durch alle Merkmalsdateien
    for merkmale_path in merkmale_paths:
        # Merkmale-Datei laden
        # Specify the delimiter as semicolon (;)
        merkmale = pd.read_csv(merkmale_path, sep=sep, low_memory=False)  # low_memory aufgrund von Kommentar, dass Datentypen der Spalten verschieden sind


        # Relevante Zeilen basierend auf Gitterzellen-ID extrahieren
        merkmale_relevant = merkmale[merkmale["GITTER_ID_100m"].isin(hauptdokument["GITTER_ID_100m"])]

        # Spalten aus merkmale_relevant auswählen, die nicht bereits in hauptdokument vorhanden sind
        # und nicht der Merge-Key sind
        merkmale_cols = [
            col for col in merkmale_relevant.columns
            if col not in hauptdokument.columns or col == "GITTER_ID_100m"
        ]

        # Zusammenfügen der Merkmalsdaten mit dem Hauptdokument,
        # nur die ausgewählten Spalten verwenden
        hauptdokument = pd.merge(
            hauptdokument,
            merkmale_relevant[merkmale_cols],
            on="GITTER_ID_100m",
            how="left"
        )

    # Kombinierte Tabelle speichern
    hauptdokument.to_csv(output_path, index=False, sep=sep)
    print(f"Kombinierte Datei gespeichert unter: {output_path}")

In [ ]:
# Den Pfad zur CSV-Datei angeben
csv_datei = '/content/noerdlingen_merged.csv'

# Die CSV-Datei laden
df = pd.read_csv(csv_datei)

# Die ersten Zeilen anzeigen (Standard: 5 Zeilen)
print(df.head())


                   GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0  CRS3035RES100mN2854400E4359000    4359050    2854450   
1  CRS3035RES100mN2854500E4358400    4358450    2854550   
2  CRS3035RES100mN2854500E4358700    4358750    2854550   
3  CRS3035RES100mN2854500E4358800    4358850    2854550   
4  CRS3035RES100mN2854500E4358900    4358950    2854550   

   werterlaeuternde_Zeichen                 geometry AnteilUeber65  \
0                       NaN  POINT (4359050 2854450)             –   
1                       NaN  POINT (4358450 2854550)             –   
2                       NaN  POINT (4358750 2854550)             –   
3                       NaN  POINT (4358850 2854550)         13,64   
4                       NaN  POINT (4358950 2854550)             –   

  AnteilAuslaender durchschnFlaechejeBew Durchschnittsalter  Einwohner  \
0                –                 65,00              37,13          3   
1                –                   NaN              53,48          3   
2 

es soll noch ein Parameter dazu: Anzahl der Bevölkerung im Alter von unter 10 Jahren

=> dieser ist noch in Tabelle mit weiteren Altersklassen

1. alle anderen Altersklassen (Spalten) löschen

2. Anteil der Bevölkerung unter 10 berechnen

=> Anzahl unter 10 / insgesamt Bevölkerung

aus – 0 oder leere Zelle machen

In [ ]:
# Schritt 1: CSV-Datei mit Gitterzellen laden
# Annahme: CSV enthält Spalten "X" und "Y" für die Koordinaten
csv_path = "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter.csv"
gitter_df = pd.read_csv(csv_path, sep=";")

gitter_id_column = gitter_df["GITTER_ID_100m"]

print(gitter_df.head)
print("Spaltennamen vor dem Löschen:", gitter_df.columns)

<bound method NDFrame.head of                          GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0        CRS3035RES100mN2689100E4337000    4337050    2689150   
1        CRS3035RES100mN2689100E4341100    4341150    2689150   
2        CRS3035RES100mN2690800E4341200    4341250    2690850   
3        CRS3035RES100mN2691200E4341200    4341250    2691250   
4        CRS3035RES100mN2691300E4341200    4341250    2691350   
...                                 ...        ...        ...   
3088032  CRS3035RES100mN3547500E4220100    4220150    3547550   
3088033  CRS3035RES100mN3547600E4219800    4219850    3547650   
3088034  CRS3035RES100mN3547600E4219900    4219950    3547650   
3088035  CRS3035RES100mN3547600E4220000    4220050    3547650   
3088036  CRS3035RES100mN3547600E4220100    4220150    3547650   

         Insgesamt_Bevoelkerung Unter10 a10bis19 a20bis29 a30bis39 a40bis49  \
0                             4       –        –        –        –        –   
1                            11

In [ ]:
gitter_df = gitter_df.drop(columns=["a10bis19", "a20bis29", "a30bis39", "a40bis49", "a50bis59", "a60bis69", "a70bis79", "a80undaelter"])

print ("Spaltennamen nach dem Löschen:", gitter_df.columns)


Spaltennamen nach dem Löschen: Index(['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'Insgesamt_Bevoelkerung',
       'Unter10'],
      dtype='object')


In [ ]:
# Anteil der Unter-10-Jährigen berechnen
# Replace dashes with 0 and convert to numeric
gitter_df['Unter10'] = pd.to_numeric(gitter_df['Unter10'].astype(str).str.replace('–', '0'), errors='coerce')
gitter_df['Insgesamt_Bevoelkerung'] = pd.to_numeric(gitter_df['Insgesamt_Bevoelkerung'].astype(str).str.replace('–', '0'), errors='coerce')

# Calculate the percentage, handling potential division by zero
gitter_df['Anteil_unter10'] = (gitter_df['Unter10'] / gitter_df['Insgesamt_Bevoelkerung']) * 100
gitter_df['Anteil_unter10'] = gitter_df['Anteil_unter10'].fillna(0) # Replace NaN with 0 if 'Insgesamt_Bevoelkerung' is 0

In [ ]:
# Before saving, make sure 'GITTER_ID_100m' is included
# If it was dropped earlier, you need to add it back
gitter_df['GITTER_ID_100m'] = gitter_id_column

In [ ]:
gitter_df.head(10)

,GITTER_ID_100m,x_mp_100m,y_mp_100m,Insgesamt_Bevoelkerung,Unter10,Anteil_unter10
0,CRS3035RES100mN2689100E4337000,4337050,2689150,4,0,0.0
1,CRS3035RES100mN2689100E4341100,4341150,2689150,11,0,0.0
2,CRS3035RES100mN2690800E4341200,4341250,2690850,4,0,0.0
3,CRS3035RES100mN2691200E4341200,4341250,2691250,12,0,0.0
4,CRS3035RES100mN2691300E4341200,4341250,2691350,3,0,0.0
5,CRS3035RES100mN2691700E4341100,4341150,2691750,8,0,0.0
6,CRS3035RES100mN2691900E4340800,4340850,2691950,3,0,0.0
7,CRS3035RES100mN2692400E4341200,4341250,2692450,5,0,0.0
8,CRS3035RES100mN2692600E4341000,4341050,2692650,9,0,0.0
9,CRS3035RES100mN2692600E4344000,4344050,2692650,12,0,0.0


In [ ]:
# Datei speichern in Google Drive
gitter_df.to_csv('/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/Anteil_Unter10_DE.csv', index=False, sep=";")


In [ ]:
csv_path = "/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/Anteil_Unter10_DE.csv"
AnteilU10_df = pd.read_csv(csv_path, sep=";")

print(AnteilU10_df.head)


<bound method NDFrame.head of                          GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0        CRS3035RES100mN2689100E4337000    4337050    2689150   
1        CRS3035RES100mN2689100E4341100    4341150    2689150   
2        CRS3035RES100mN2690800E4341200    4341250    2690850   
3        CRS3035RES100mN2691200E4341200    4341250    2691250   
4        CRS3035RES100mN2691300E4341200    4341250    2691350   
...                                 ...        ...        ...   
3088032  CRS3035RES100mN3547500E4220100    4220150    3547550   
3088033  CRS3035RES100mN3547600E4219800    4219850    3547650   
3088034  CRS3035RES100mN3547600E4219900    4219950    3547650   
3088035  CRS3035RES100mN3547600E4220000    4220050    3547650   
3088036  CRS3035RES100mN3547600E4220100    4220150    3547650   

         Insgesamt_Bevoelkerung  Unter10  Anteil_unter10  
0                             4        0             0.0  
1                            11        0             0.0  
2           

In [ ]:
# Beispiel: Hauptdokumente und Merkmalsdateien
noerdlingen_path = "/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/gitterzellen_noerdlingen_clean.csv"
kempten_path = "/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/gitterzellen_kempten_clean.csv"
ingolstadt_path = "/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/gitterzellen_Ingolstadt_clean.csv"

# Pfade zu den Merkmalsdateien (z. B. Durchschnittsalter, Einkommen)
merkmale_paths = [
    "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Anteil_ab_65-jaehrige_in_Gitterzellen/Zensus2022_Anteil_ueber_65_100m-Gitter.csv",
    "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Auslaenderanteil_in_Gitterzellen/Zensus2022_Anteil_Auslaender_100m-Gitter.csv",
    "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Durchschnittliche_Wohnflaeche_je_Bewohner_in_Gitterzellen/Zensus2022_Durchschn_Flaeche_je_Bewohner_100m-Gitter.csv",
    "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Durchschnittsalter_in_Gitterzellen/Zensus2022_Durchschnittsalter_100m-Gitter.csv",
    "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Zensus2022_Bevoelkerungszahl/Zensus2022_Bevoelkerungszahl_100m-Gitter.csv",
    "/content/drive/MyDrive/Daten BA /Zensus_2022-Daten-Excel/Zensus2022_Durchschn_Nettokaltmiete/Zensus2022_Durchschn_Nettokaltmiete_100m-Gitter.csv",
    "/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/Anteil_Unter10_DE.csv"

]

# Kombinieren für jede Stadt
kombiniere_daten(noerdlingen_path, merkmale_paths, "NOERDLINGEN_neu_merged.csv", sep=";")
kombiniere_daten(kempten_path, merkmale_paths, "KEMPTEN_neu_merged.csv", sep=";")
kombiniere_daten(ingolstadt_path, merkmale_paths, "INGOLSTADT_neu_merged.csv", sep=";")


Kombinierte Datei gespeichert unter: NOERDLINGEN_neu_merged.csv
Kombinierte Datei gespeichert unter: KEMPTEN_neu_merged.csv
Kombinierte Datei gespeichert unter: INGOLSTADT_neu_merged.csv


In [ ]:
import pandas as pd
import pandas as pd

# CSV-Datei lesen mit expliziter Angabe von Encoding und Datentyp ( encoding='utf-8', dtype={'Spaltenname': str})
Noerdlingen_df = pd.read_csv('/content/NOERDLINGEN_neu_merged.csv', sep=';')
Kempten_df = pd.read_csv('/content/KEMPTEN_neu_merged.csv', sep=';')
Ingolstadt_df = pd.read_csv('/content/INGOLSTADT_neu_merged.csv', sep=';')

print(Noerdlingen_df.head(5))
print(Kempten_df.head(5))
print(Ingolstadt_df.head(5))

                   GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0  CRS3035RES100mN2854400E4359000    4359050    2854450   
1  CRS3035RES100mN2854500E4358400    4358450    2854550   
2  CRS3035RES100mN2854500E4358700    4358750    2854550   
3  CRS3035RES100mN2854500E4358800    4358850    2854550   
4  CRS3035RES100mN2854500E4358900    4358950    2854550   

                  geometry AnteilUeber65 werterlaeuternde_Zeichen  \
0  POINT (4359050 2854450)             –                      NaN   
1  POINT (4358450 2854550)             –                      NaN   
2  POINT (4358750 2854550)             –                      NaN   
3  POINT (4358850 2854550)         13,64                 KLAMMERN   
4  POINT (4358950 2854550)             –                      NaN   

  AnteilAuslaender durchschnFlaechejeBew Durchschnittsalter  Einwohner  \
0                –                 65,00              37,13          3   
1                –                   NaN              53,48          3   
2       

In [ ]:
Noerdlingen_df = Noerdlingen_df.drop(columns=["werterlaeuternde_Zeichen"])
Kempten_df = Kempten_df.drop(columns=["werterlaeuternde_Zeichen"])
Ingolstadt_df = Ingolstadt_df.drop(columns=["werterlaeuternde_Zeichen"])

print(Noerdlingen_df.head(5))
print(Kempten_df.head(5))
print(Ingolstadt_df.head(5))

                   GITTER_ID_100m  x_mp_100m  y_mp_100m  \
0  CRS3035RES100mN2854400E4359000    4359050    2854450   
1  CRS3035RES100mN2854500E4358400    4358450    2854550   
2  CRS3035RES100mN2854500E4358700    4358750    2854550   
3  CRS3035RES100mN2854500E4358800    4358850    2854550   
4  CRS3035RES100mN2854500E4358900    4358950    2854550   

                  geometry AnteilUeber65 AnteilAuslaender  \
0  POINT (4359050 2854450)             –                –   
1  POINT (4358450 2854550)             –                –   
2  POINT (4358750 2854550)             –                –   
3  POINT (4358850 2854550)         13,64                –   
4  POINT (4358950 2854550)             –                –   

  durchschnFlaechejeBew Durchschnittsalter  Einwohner durchschnMieteQM  \
0                 65,00              37,13          3              NaN   
1                   NaN              53,48          3              NaN   
2                 39,04              57,78          6   

In [ ]:
Noerdlingen_df.to_csv('/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/NOERDLINGEN_neu_merged.csv', index=False, sep=";")
Kempten_df.to_csv('/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/KEMPTEN_neu_merged.csv', index=False, sep=";")
Ingolstadt_df.to_csv('/content/drive/MyDrive/Daten BA /erstellte Dateien mit colab/INGOLSTADT_neu_merged.csv', index=False, sep=";")

Tabellen jetzt final in Drive gespeichert unter NOERDLINGEN_neu_merged.csv,...